In [1]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.cross_validation import train_test_split
import numpy as np

In [2]:
df = pd.read_csv("train.csv", index_col="row_id")

max_coordinate = 2

subset = df.loc[(df.x < max_coordinate) & (df.y < max_coordinate), :]
del df

In [3]:
X = subset.iloc[:, :-1].values # remove place_id label
y = np.squeeze(subset.iloc[:, -1:].values) # extract place_id label

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

def split_X(m): # split data matrix with x&y columns and into a matrix with accuracy and time columns
    return m[:,:2], m[:,2:]

X_train_xy, X_train_ac = split_X(X_train)
X_test_xy, X_test_ac = split_X(X_test)

In [4]:
neigh = NearestNeighbors(n_jobs=-1)
%time neigh.fit(X_train_xy, y_train)

CPU times: user 2.09 s, sys: 19.3 ms, total: 2.11 s
Wall time: 2.12 s


NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=-1, n_neighbors=5, p=2, radius=1.0)

In [11]:
%time distances, neighbor_indicies = neigh.kneighbors(X_test_xy, n_neighbors=50)

CPU times: user 9.9 s, sys: 591 ms, total: 10.5 s
Wall time: 3.25 s


In [85]:
# create predictions
def predict():
    return [y_train[i[:3]] for i in neighbor_indicies]
        
%time predictions = predict()
predictions[:5]

CPU times: user 235 ms, sys: 13.1 ms, total: 248 ms
Wall time: 247 ms


[array([5356564080, 5356564080, 2055909431]),
 array([1304193568, 3884608029, 4982492326]),
 array([8624673382, 9551909561, 9551909561]),
 array([3380753307, 9649118066, 9649118066]),
 array([8180961305, 7677441016, 8180961305])]

In [74]:
y_test[:5]

array([8518596442, 1218151301, 9551909561, 9649118066, 1285016904])

In [68]:
def mean_average_precision3(y_true, y_test):
    

array([8518596442, 1218151301, 9551909561, 9649118066, 1285016904])